In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
!pip install -q d2l

from d2l import torch as d2l
import torch
from torch import nn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.9/58.9 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.7/111.7 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 69.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 61.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 59.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.1/34.1 MB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.3/98.3 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.0/125.0 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.0/95.0 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 51.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the sour

In [2]:
import os
! git clone https://git.wur.nl/bioinformatics/grs34806-deep-learning-project-data.git
os.chdir("grs34806-deep-learning-project-data")

Cloning into 'grs34806-deep-learning-project-data'...
remote: Enumerating objects: 21, done.
remote: Total 21 (delta 0), reused 0 (delta 0), pack-reused 21 (from 1)
Receiving objects: 100% (21/21), 8.74 MiB | 5.40 MiB/s, done.


In [3]:
def read_files(seqfile,posfile):
# seqfile: file with sequences
# posfile: file with positive cases (annotated with function)

  with open(seqfile) as seq_handle:
    datalist = []
    for line in seq_handle:
      datalist.append(line.split()[1])

  with open(posfile) as pos_handle:
    labellist = [0] * len(datalist)
    for line in pos_handle:
      labellist[int(line[3:]) - 1] = 1

  return datalist, labellist

In [4]:
all_data = read_files('/content/grs34806-deep-learning-project-data/len100_200_n1000.seq', '/content/grs34806-deep-learning-project-data/len100_200_n1000.pos')

In [5]:
df = pd.DataFrame(data = zip(all_data[0], all_data[1]), columns = ['Sequence', 'label'])

In [6]:
X_train, X_test, y_train, y_test = train_test_split(
    all_data[0], all_data[1], test_size=0.33, random_state=42)

In [7]:
def load_data(batch_size, num_steps, dataset):
  mapaa2num = {aa: i for (i, aa)
                in enumerate(list("ACDEFGHIKLMNPQRSTVWY"))}
  seq,lab = dataset
  seq = tokenize(seq, mapaa2num)
  seq_array = build_seq_array(seq, num_steps)
  data_arrays = (seq_array, torch.tensor(lab))
  data_iter = d2l.load_array(data_arrays, batch_size)
  return data_iter

def tokenize(dat, map2num,non_aa_num=20):
  seq = []
  for count, i in enumerate(dat):
    seq.append([map2num.get(j,non_aa_num) for j in list(i)])
  return seq

def build_seq_array(lines, num_steps, non_aa_num=20):
  array = torch.tensor([
  truncate_pad(l, num_steps, non_aa_num) for l in lines])
  return array

def truncate_pad(line, num_steps, padding_token):
  if len(line) > num_steps:
    return line[:num_steps] # Truncate
  return line + [padding_token] * (num_steps - len(line)) # Pad


In [10]:
! pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.6/383.6 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 21.8 MB/s eta 0:00:00


In [56]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import ParameterGrid
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
from d2l import torch as d2l


# === Deep CNN Model with nn.Sequential ===
class DeepProteinCNN(nn.Module):
    def __init__(self, vocab_size, embed_dim, out_channels, kernel_size, dropout_rate, num_classes=2):
        super(DeepProteinCNN, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=vocab_size - 1)

        self.conv_layers = nn.Sequential(
            nn.Conv1d(embed_dim, out_channels, kernel_size, padding="same"),
            nn.ReLU(),
            nn.MaxPool1d(2),

            nn.Conv1d(out_channels, out_channels * 2, kernel_size, padding="same"),
            nn.ReLU(),
            nn.MaxPool1d(2),

            nn.Conv1d(out_channels * 2, out_channels * 4, kernel_size, padding="same"),
            nn.ReLU(),
            nn.AdaptiveMaxPool1d(1)
        )

        self.dropout = nn.Dropout(dropout_rate)
        self.fc = nn.Linear(out_channels * 4, num_classes)

    def forward(self, x):
        x = self.embedding(x)             # (batch, seq_len, embed_dim)
        x = x.permute(0, 2, 1)            # (batch, embed_dim, seq_len)
        x = self.conv_layers(x)          # (batch, out_channels * 4, 1)
        x = x.squeeze(-1)                # (batch, out_channels * 4)
        x = self.dropout(x)
        return self.fc(x)

# === Training & Evaluation ===
def train_model(model, optimizer, criterion, loader):
    model.train()
    for batch_X, batch_y in loader:
        optimizer.zero_grad()
        preds = model(batch_X)
        loss = criterion(preds, batch_y)
        loss.backward()
        optimizer.step()

def evaluate_model(model, loader):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for batch_X, batch_y in loader:
            preds = model(batch_X).argmax(dim=1)
            correct += (preds == batch_y).sum().item()
            total += batch_y.size(0)
    return correct / total

# === Grid Search ===
param_grid = {
    'kernel_size': [5, 8],
    'out_channels': [32],
    'embed_dim': [20, 50],
    'dropout_rate': [0.2, 0.4, 0.6],
    'lr': [1e-3, 1e-4, 1e-5]
}

grid = list(ParameterGrid(param_grid))

# === Load Your Dataset ===
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


batch_size = 64
num_steps = 140  # fixed length of sequence

train_loader = load_data(batch_size, num_steps, (X_train, y_train))
test_loader = load_data(batch_size, num_steps, (X_test, y_test))

# === Grid Search Loop ===
best_acc = 0
best_params = None

for params in grid:
    model = DeepProteinCNN(
        vocab_size=21,
        embed_dim=params['embed_dim'],
        out_channels=params['out_channels'],
        kernel_size=params['kernel_size'],
        dropout_rate=params['dropout_rate']
    )
    optimizer = optim.Adam(model.parameters(), lr=params['lr'])
    criterion = nn.CrossEntropyLoss()

    for epoch in range(5):
        train_model(model, optimizer, criterion, train_loader)

    acc = evaluate_model(model, test_loader)
    print(f"Params: {params}, Accuracy: {acc:.4f}")

    if acc > best_acc:
        best_acc = acc
        best_params = params

print("\n🔥 Best Params:")
print(best_params)
print(f"✅ Best Accuracy: {best_acc:.4f}")

Params: {'dropout_rate': 0.2, 'embed_dim': 20, 'kernel_size': 5, 'lr': 0.001, 'out_channels': 32}, Accuracy: 0.9636
Params: {'dropout_rate': 0.2, 'embed_dim': 20, 'kernel_size': 5, 'lr': 0.0001, 'out_channels': 32}, Accuracy: 0.5364
Params: {'dropout_rate': 0.2, 'embed_dim': 20, 'kernel_size': 5, 'lr': 1e-05, 'out_channels': 32}, Accuracy: 0.4788
Params: {'dropout_rate': 0.2, 'embed_dim': 20, 'kernel_size': 8, 'lr': 0.001, 'out_channels': 32}, Accuracy: 0.9515
Params: {'dropout_rate': 0.2, 'embed_dim': 20, 'kernel_size': 8, 'lr': 0.0001, 'out_channels': 32}, Accuracy: 0.5515
Params: {'dropout_rate': 0.2, 'embed_dim': 20, 'kernel_size': 8, 'lr': 1e-05, 'out_channels': 32}, Accuracy: 0.5212
Params: {'dropout_rate': 0.2, 'embed_dim': 50, 'kernel_size': 5, 'lr': 0.001, 'out_channels': 32}, Accuracy: 0.9545
Params: {'dropout_rate': 0.2, 'embed_dim': 50, 'kernel_size': 5, 'lr': 0.0001, 'out_channels': 32}, Accuracy: 0.5545
Params: {'dropout_rate': 0.2, 'embed_dim': 50, 'kernel_size': 5, 'lr'